In this lab, you are asked to evaluate Chain-of-Thought prompt and self-consistency performance on math reasoning dataset -- SVAMP (https://huggingface.co/datasets/ChilleD/SVAMP/viewer/default/test).


In [1]:
!pip install accelerate
!pip install bitsandbytes
!pip install transformers
!pip install SentencePiece
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 6.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 31.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 81.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00


In [2]:
!nvidia-smi

Fri Nov 17 03:36:38 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Load Dataset
Load the SVAMP dataset. Similar to last assignment, shoule be easy for you.

In [3]:
#Q1 Load the SVAMP dataset (1 point)
from datasets import load_dataset
dataset = load_dataset("ChilleD/SVAMP")
print(dataset.keys())

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

dict_keys(['train', 'test'])


In [4]:
# Prepare the input data and ground_truth
questions = []
answers = []

count = 0
for example in dataset["test"]:   # we only use first 100 samples for evaluation
  #Q2: exact questions and answers from the original dataset. Please check the original dataset carefully!!! (1 point)
  questions.append(example["Body"]+" "+example["Question"])
  answers.append(example["Answer"])
  count += 1
  if count == 100:
    break

print(f"the number of questions: {len(questions)}")
print(f"the number of answers: {len(answers)}")
print(questions[0])
print(answers[0])

the number of questions: 100
the number of answers: 100
Mary is baking a cake. The recipe calls for 6 cups of flour 8 cups of sugar and 7 cups of salt. She already put in 5 cups of flour. How many more cups of sugar than cups of salt does she need to add now?
1.0


In [5]:
import os
import torch
import torch.nn as nn
import bitsandbytes as bnb
import accelerate
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM
from transformers import LlamaTokenizer

#Q3: load the model "meta-math/MetaMath-7B-V1.0" with 8bit setting (1 point)
# the model name and loading setting
load_in_8bit = True
model_name = "meta-math/MetaMath-7B-V1.0"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    load_in_8bit=load_in_8bit,
    device_map="auto"
)
tokenizer = LlamaTokenizer.from_pretrained(model_name)

(…)etaMath-7B-V1.0/resolve/main/config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

(…)esolve/main/pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00003.bin:   0%|          | 0.00/9.88G [00:00<?, ?B/s]

pytorch_model-00002-of-00003.bin:   0%|          | 0.00/9.89G [00:00<?, ?B/s]

pytorch_model-00003-of-00003.bin:   0%|          | 0.00/7.18G [00:00<?, ?B/s]

...

(…)V1.0/resolve/main/generation_config.json:   0%|          | 0.00/183 [00:00<?, ?B/s]

(…)-V1.0/resolve/main/tokenizer_config.json:   0%|          | 0.00/748 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

(…)h-7B-V1.0/resolve/main/added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

(…)1.0/resolve/main/special_tokens_map.json:   0%|          | 0.00/96.0 [00:00<?, ?B/s]

In [6]:
# create the function to generate response with LLMs
# you can refer to the tutorial material
def response_gen(input_ids):
  temperature=0.7
  with torch.no_grad():
      generation_output = model.generate(
          input_ids=input_ids,
          temperature=temperature,
          top_p = 1.0,
          do_sample=True,
          return_dict_in_generate=True,
          max_new_tokens=256,
      )
  s = generation_output.sequences[0][len(input_ids[0]):]
  output = tokenizer.decode(s)
  return output

In [7]:
from transformers import GenerationConfig

# Q4: Prepare query prompt and generate response, please use CoT prompt f"{question} Please think step by step." (1 point)
predictions = []
for question in questions[:10]:
  print("Questions:", question)
  prompt = f"{question} Please think step by step."
  input = tokenizer(prompt, return_tensors='pt')
  input_ids = input["input_ids"].to("cuda")

  output = response_gen(input_ids)
  print("Prediction:", output)
  predictions.append(output)

Questions: Mary is baking a cake. The recipe calls for 6 cups of flour 8 cups of sugar and 7 cups of salt. She already put in 5 cups of flour. How many more cups of sugar than cups of salt does she need to add now?
Prediction: 
The recipe calls for 6 cups of flour, and she already put in 5 cups, so she needs 6 - 5 = 1 more cup of flour.
The recipe calls for 8 cups of sugar, so she needs to add 8 - 1 = 7 more cups of sugar.
The recipe calls for 7 cups of salt, so she needs to add 7 - 7 = 0 more cups of salt.
Therefore, she needs to add 7 - 0 = 7 more cups of sugar than cups of salt.
#### 7
The answer is: 7</s>
Questions: Paul got a box of some crayons for his birthday. During the school year he gave 52 crayons to his friends while he lost 535 crayons. If he only had 492 crayons left How many crayons had been lost or given away?
Prediction: 

During the school year, John gave away 52 crayons and lost 535 crayons.
So, the total number of crayons lost or given away is 52 + 535 = 587 crayon

In [8]:
import re
def extract_answer_number(sentence: str) -> float:
  sentence = sentence.replace(',', '')
  pred = [s for s in re.findall(r'-?\d+\.?\d*', sentence)]
  if not pred:
      return float('inf')
  pred_answer = float(pred[-1])
  if isinstance(pred_answer, str):
      try:
          pred_answer = float(pred_answer)
      except ValueError as e:
          pred_answer = float('inf')
  return pred_answer

In [9]:
def acc_cal(answers, predictions):
  correct = 0
  for i in range(len(answers)):
    pred_number = extract_answer_number(predictions[i])
    if pred_number == answers[i]:
      correct += 1
  return correct/len(answers)

In [11]:
acc_cal(answers[:10], predictions[:10])

0.7

#CoT + Self-Consistency
Q5: please write your code to do CoT + Self-consistency with following 2 steps

Step 1: for each question, query the model for n times, n can be 5 or 10. (3 points)

Step 2: for each question, count the frequency of the predicted answers, and choose the most frequent one as the answer (3 points)

In [15]:
from collections import Counter
n_times = 5
predictions_sc = []

for question in questions[:10]:
  print("Question:", question)
  prompt = f"{question} Please think step by step."
  input = tokenizer(prompt, return_tensors='pt')
  input_ids = input["input_ids"].to("cuda")
  i_predictions = []
  for i in range(n_times):
    output = response_gen(input_ids)
    print("Prediction "+str(i+1)+":", output)
    answer = extract_answer_number(output)
    i_predictions.append(str(answer))
  counter = Counter(i_predictions)
  most_freq_pred, count = counter.most_common(1)[0]
  print(f"The most common prediction is {most_freq_pred} with a count of {count}.")
  predictions_sc.append(most_freq_pred)

Question: Mary is baking a cake. The recipe calls for 6 cups of flour 8 cups of sugar and 7 cups of salt. She already put in 5 cups of flour. How many more cups of sugar than cups of salt does she need to add now?
Prediction 1: 

The recipe calls for 6 cups of flour, and she already put in 5 cups, so she needs to add 6 - 5 = 1 more cup of flour.
The recipe calls for 8 cups of sugar, and she needs to add 1 more cup of sugar than flour, so she needs to add 8 + 1 = 9 cups of sugar.
The recipe calls for 7 cups of salt, and she needs to add 1 more cup of salt than sugar, so she needs to add 7 + 1 = 8 cups of salt.
Therefore, she needs to add 9 - 8 = 1 more cup of sugar than cups of salt.
#### 1
The answer is: 1</s>
Prediction 2: 


Mary already put in 5 cups of flour, so she still needs 6 - 5 = 1 more cup of flour.
The recipe calls for 8 cups of sugar, so she still needs 8 - 5 = 3 more cups of sugar.
The recipe calls for 7 cups of salt, so she still needs 7 - 5 = 2 more cups of salt.
Theref

In [16]:
acc_cal(answers[:10], predictions_sc[:10])

0.8